In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pickle as pkl
from sklearn.base import BaseEstimator
import os
import numpy as np
import pickle as pkl
from data_generation import GaussianDataGenerator
from algorithms import *
from sklearn.model_selection import GridSearchCV
import warnings

warnings.filterwarnings("ignore")

In [2]:
n = 600
m = 20
p = 1000
noise_level = 0.1
trial_id = 0


Data_Geneartor = GaussianDataGenerator(p, n, m, noise_level, trial_id)
true_signal, dictionary, true_indices, true_coefficients, perturbed_signal = Data_Geneartor.shuffle()

temp_SignalAtomBagging = SignalAtomBagging(10)
s_bag, phi_bag, col_idx_bag = temp_SignalAtomBagging.fit(dictionary, perturbed_signal)
temp_BOMP = BOMP(Bag_lst=[10,20,30,40,50,60,70,80,90,100],K_lst=[10,20,30,40,50,60,70,80,90,100])
temp_BOMP.fit(phi =dictionary, s = perturbed_signal)

[[1.00000000e+01 1.00000000e+02 4.60713504e-03]
 [2.00000000e+01 1.00000000e+02 4.09476165e-03]
 [3.00000000e+01 1.00000000e+02 3.94438765e-03]
 [4.00000000e+01 1.00000000e+02 3.96855847e-03]
 [5.00000000e+01 1.00000000e+02 3.86261174e-03]
 [6.00000000e+01 1.00000000e+02 3.80957081e-03]
 [7.00000000e+01 1.00000000e+02 3.78992485e-03]
 [8.00000000e+01 1.00000000e+02 3.77940520e-03]
 [9.00000000e+01 1.00000000e+02 3.77117551e-03]
 [1.00000000e+02 1.00000000e+02 3.75116623e-03]]


(array([ 1.60686510e-02,  7.82948284e-02, -3.81555113e-01, -1.59696964e-01,
        -3.95337894e-02, -2.05366973e-01,  1.13510060e-01, -4.89289853e-01,
         1.87272120e-01,  3.10157760e-02, -3.63495818e-01,  2.07028026e-03,
        -3.16872316e-01, -1.31544036e-01, -1.32268870e-01,  2.34356429e-01,
        -3.70748198e-01, -2.09993476e-01, -1.12917090e-01,  9.25174602e-02,
        -2.04599997e-01,  1.60316306e-01,  4.57111534e-02, -1.91083748e-01,
        -2.43974186e-01,  4.13213516e-02, -1.55419694e-01, -5.07225545e-02,
         1.75352880e-01,  6.68561187e-02, -1.57126483e-01, -1.34236089e-01,
         3.40989343e-03, -2.00401056e-01, -1.27875164e-02, -1.27153958e-01,
         2.91358151e-01,  1.14445302e-02,  1.21170731e-01, -1.74148008e-01,
        -5.95305834e-02, -3.13868318e-01,  2.04815174e-02,  8.31956864e-02,
         2.08611557e-01,  1.46216761e-01,  2.03619587e-02, -1.56442407e-01,
         3.57312382e-03, -4.02491899e-01,  4.64422914e-01,  3.25307469e-01,
        -9.0

In [3]:
def run_trials_npm_multi_noise_lvl(n, p, m, noise_level_lst, model_name, fixed_params, param_grid, cv_num, trial_num):
    # get the model

    if model_name == "BOMP": 
        model = BOMP(**fixed_params)

    
    res_log_npm = {
        'parameters': {'n': n, 'p': p, 'm': m, 'noise_level_lst': noise_level_lst, 'model_name': model_name, 'cv_num': cv_num, 'trial_num': trial_num, 'param_grid': param_grid, 'fixed_params': fixed_params},
        'noise_level_lowest_MSE': [],
        'log': []
    }
    print(f"Running trials for n = {n}, p = {p}, m = {m}")
    for noise_level in noise_level_lst:
        print("Cross validating alpha under noise level: ", noise_level)
        trials_loweset_MSE_temp = []
        for trial_id in range(trial_num):
            Data_Geneartor = GaussianDataGenerator(p, n, m, noise_level, trial_id)
            true_signal, dictionary, true_indices, true_coefficients, perturbed_signal = Data_Geneartor.shuffle()
            gs = GridSearchCV(model, param_grid, cv=cv_num, scoring='neg_mean_squared_error', n_jobs=-1, verbose=0)
            gs.fit(dictionary, perturbed_signal)
            cv_err_lst = -gs.cv_results_['mean_test_score']
            param_lst = gs.cv_results_['params']
            lowest_error = np.min(cv_err_lst)
            trials_loweset_MSE_temp.append(lowest_error)
            best_params = gs.best_params_
            reslog_one_trial = {'noise_level': noise_level, 'trial': trial_id, 'cv_error_lst': cv_err_lst, 
                            'lowest_error': lowest_error, 'best_params': best_params, 'param_lst': param_lst}
            res_log_npm['log'].append(reslog_one_trial)
            print("Trial: ", trial_id, " Best params: ", best_params, " Lowest Error: ", lowest_error)
        res_log_npm['noise_level_lowest_MSE'].append(np.mean(trials_loweset_MSE_temp))
        print("Noise level: ", noise_level, " Avg Lowest MSE: ", np.mean(trials_loweset_MSE_temp))
    return res_log_npm